<a href="https://colab.research.google.com/github/jrodriguezru/ML-Interpretability/blob/main/Notebook_Interpretabilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpretabilidad de Modelos de Machine Learning

Los modelos de machine learning normalmente son contemplados como "cajas negras" en donde no se sabe con certeza la importancia y el peso de cada variable en el resultado del modelo tanto para observaciones individuales, como para dar una explicación general del modelo.

Este notebook está diseñado para dar una aproximación a la librería *shap*, la cual provee múltiples *Explainers* para diferentes tipos de modelos de machine learning y con la que se puede hacer diferentes tipos de gráficas que explican dichos modelos.

Se explorará la librería usando dos modelos que se adaptan a dos datasets diferentes para mostrar la importancia de ajustar las escalas para que la interpretabilidad sea desarrollada correctamente.

Este notebook usa *Keras - TensorFlow* para lo cual, asegúrese que ha seleccionado **GPU T4** como **tipo de entorno de ejecución** en la configuración de la sesión de Google Colab.

**Este notebook está diseñado con funcionalidades de autocalificación. Para el correcto funcionamiento, solo escriba su código  dentro de los espacios marcados para ello.**

## Contenidos

- [0 - Configuración de Google Colab](#0)
- [1 - Paquetes](#1)
- [2 - Funciones Auxiliares](#2)
- [3 - Modelo de clasificación](#3)
    - [3.1 - Exploración Inicial del Dataset y Pre-Procesamiento](#3-1)
    - [3.2 - Construcción y Entrenamiento del Modelo](#3-2)
    - [3.3 - SHAP y Explainers](#3-3)
    - [3.4 - Gráficas Locales](#3-4)
    - [3.5 - Gráficas Globales](#3-5)
- [4 - Modelo de Regresión](#4)
    - [4.1 - Exploración Inicial del Dataset y Pre-Procesamiento](#4-1)
    - [4.2 - Construcción y Entrenamiento del Modelo](#4-2)
    - [4.3 - SHAP y Explainers](#4-3)
    - [4.4 - Gráficas Locales](#4-4)
    - [4.5 - Gráficas Globales](#4-5)
- [5 - Hazlo Tu Mismo (Ungraded)](#5)
- [6 - Bibliografía](#6)



<a name='0'></a>
## 0 - Configuración de Google Colab

Asegúrese que la carpeta que se incluye con este notebook sea guardada dentro de **Mi Unidad** en su cuenta de Google Drive. Ejecute la siguiente celda y acceda a su cuenta, dando los permisos necesarios. Esto hará que este notebook tenga acceso a los archivos necesarios para la autocalificación.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd drive/MyDrive/labInterpretability

<a name='1'></a>
## 1 - Paquetes Necesarios

La siguiente celda importa todas las librerías necesarias para que este notebook funcione correctamente.

In [ ]:
%pip install ipywidgets
%pip install colorama
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, Embedding, Flatten, Input, concatenate
from keras.models import Model
import keras
import ipywidgets as widgets
from IPython.display import display
import pytest
import tests
from question_bank import display_question

np.random.seed(1)
tf.random.set_seed(2)

La siguiente celda configura TensorFlow y Keras para que se use la GPU disponible. Asegúrese que ha seleccionado **GPU T4** como **tipo de entorno de ejecución** en la configuración de la sesión de Google Colab.

In [ ]:
# Check if a GPU is available
gpu_available = tf.config.list_physical_devices('GPU')

if gpu_available:
    print("GPU está disponible. Usando GPU.")
    # Set TensorFlow to use the GPU
    tf.config.experimental.set_visible_devices(gpu_available[0], 'GPU')
else:
    print("No se encontró GPU. Usando CPU.")

<a name='2'></a>
## 2 - Funciones Auxiliares

Se usará una función auxiliar para todos los modelos:
- Una función que encapsula un paso del preprocesamiento de los datos, más especificamente aplicando normalización.

In [ ]:
def normalize(x_train):
  nn_preprocessor = MinMaxScaler(feature_range=(-1, 1))
  X_train = nn_preprocessor.fit_transform(x_train)
  return X_train

<a name='3'></a>
## 3 - Modelo de Clasificación

El dataset para esta sección relaciona algunos datos de vehículos y sus dueños y lo relaciona con el número de veces que el dueño ha hecho reclamos al seguro.

In [ ]:
df = pd.read_parquet('df.parquet')
df

In [ ]:
# Almacenamos los nombres de las columnas en las variables x y y
y, x = df.columns[-1], list(df.columns[:-1])

<a name='3-1'></a>
### 3.1 - Exploración del Dataset y Pre-Procesamiento

Una buena práctica antes de iniciar con la construcción de modelos, es hacer un breve análisis del dataset. La siguiente celda da un resumen de los datos del dataframe.

In [ ]:
df.describe()

El primer paso del preprocesamiento es dividir los datos en train y test. Para esto usaremos la función `train_test_split`incluida en la librería `sklearn`. Haremos una división 90% train, 10% test.



In [ ]:
train, test = train_test_split(df, test_size=0.1, random_state=30)

In [ ]:
x_train = train.drop('claim_nb', axis=1)
y_train = train['claim_nb']
x_test = test.drop('claim_nb', axis=1)
y_test = test['claim_nb']

El siguiente paso es hacer una normalización de los datos, para asegurarnos que los valores de todas las variables estén dentro del intervalo $(-1, 1)$. Esto ayuda con la estabilidad numérica y para prevenir que algunos pesos sean demasiado grandes o demasiado pequeños.

La siguiente celda aplica la normalización al train set usando la función auxiliar que definimos anteriormente.

In [ ]:
X_train = normalize(x_train)

<a name='3-2'></a>
### 3.2 - Construcción y entrenamiento del modelo.

La configuración del modelo para este dataset es la siguiente:
- Primera capa oculta con 64 neuronas
- Tres capas ocultas con 32 neuronas

Las capas ocultas tienen función de activación `tanh`
La capa de salida tiene función de activación `exponential`

Se usará *Adam* como optimizador y la función de pérdida `poisson`

In [ ]:
# Define the input layer
input_layer = Input(shape=(x_train.shape[1],)) # Adjust input shape to match the data's features

# Add hidden layers
hidden_layer_1 = Dense(64, activation='tanh')(input_layer) # Example number of neurons
hidden_layer_2 = Dense(32, activation='tanh')(hidden_layer_1) # Example additional layer
hidden_layer_3 = Dense(32, activation='tanh')(hidden_layer_2) # Layer 3
hidden_layer_4 = Dense(32, activation='tanh')(hidden_layer_3) # Layer 4

# Add the output layer
output_layer = Dense(1, activation='exponential')(hidden_layer_4)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', # Specify optimizer
              loss='poisson')

# Print the model summary
model.summary()

Ahora se entrena el modelo.

Sin embargo, para asegurarnos que la autocalificabilidad funcione correctamente, se va a cargar un modelo pre-entrenado. En caso de querer entrenar el modelo, basta con comentar la segunda linea y des-comentar la primera linea de la siguiente celda.

In [ ]:
# model.fit(X_train, y_train, epochs=5)
model = keras.saving.load_model("model1.keras")

<a name='3-3'></a>
### 3.3 - SHAP y Explainers

En este notebook se describen los métodos de interpretabilidad proporcionados por la librería SHAP. Esta librería se basa en la teoría de juegos cooperativo, en donde a cada jugador se le asigna un valor de su contribución al resultado final del juego.

En Aprendizaje de Máquina, cada jugador corresponde a cada feature o componente de la variable de entrada y el resultado final del juego, sería la predicción del modelo.

La teoría de SHAP propone que la forma "justa" de asignar las contribuciones de cada componente es aquella que cumpla los sigueintes axiomas.

Sean $M={1, \dots, p}$ los componentes de la variable de entrada, con $p\in\mathbb{N},p\geq1$ y sea $c$ la predicción del modelo. Supongamos que la predicción para un subconjujnto de features $\mathcal{L} \subseteq M$ es dada por la función:
$$v: \mathcal{L} \rightarrow v(\mathcal{L}) \in \mathbb{R}$$

Luego los axiomas son los siguentes:

1. Eficiencia: La suma de todas las contribuciones, más un valor base, es igual a la contribución total. Esto es, $v(M) = \sum_{i=0}^p \phi_j^{(v)}$, donde $\phi_0^{(v)}$ denota el valor base, el cual puede ser igual a $0$.
2. Simetría: Si $v(\mathcal{L}\cup \{j\})=v(\mathcal{L}\cup \{k\})$ para todo $\mathcal{L} \subseteq M \backslash \{j,k\}$, entonces $\phi_j = \phi_k$. Es decir, si para todo subconjunto de features posible que no incluya a los features $j$ y $k$, (matemáticamente, cualquier subconjunto del conjunto de features quitando los features $j$ y $k$), el valor de la contribución total del subconjunto incluyendo a $j$ es igual a aquel que incluye a $k$.
3. Jugador nulo: Si $v(\mathcal{L}\cup\{j\}) = v(\mathcal{L})$, para toda agrupación de features $\mathcal{L}\subseteq M \backslash\{j\}$, entonces $\phi_j=0$. Es decir, si para todo subconjunto de features, la contribución total no cambia al agregar al feature $j$, entonces la contribución del feature $j$ es igual a 0.
4. Linealidad: Considere dos modelos con funciones de contribución total $v$ y $w$. Entonces, $\phi_j^{(v+w)}=\phi_j^{(v)} + \phi_j^{(w)}$ y también $\phi_j^{(\alpha v)} = \alpha\phi_j^{(v)}$, para todo $1\leq j \leq p$ y $\alpha \in \mathbb{R}$.

Los valores de SHAP son precisamente aquellos números reales que cumplen estos axiomas y están dados por la siguiente fórmula:

$$ \phi_j = \phi_j^{(v)} = \sum_{\mathcal{L}\subseteq M \backslash \{j\}} \frac{|\mathcal{L}|!(p-|\mathcal{L}|-1)!}{p!}\left[v\left(\mathcal{L}\cup \{j\} \right)- v\left(\mathcal{L}\right)\right] $$

El primer factor es conocido como el peso de Shapley y el segundo factor es precisamente la contribución del feature $j$.

Esta fórmula, se puede calcular de forma equivalente usando permutaciones mediante la siguiente fórmula:

$$ \phi_j = \phi_j^{(v)} = \frac{1}{p!} \sum_{\pi \in S_p} \left[v\left(\mathcal{L}_\pi \cup \{j\} \right)- v\left(\mathcal{L}_\pi \right)\right] $$

Donde $\mathcal{L}_\pi \in \mathcal{M} \backslash \{j\}$ denota el subconjunto de todos los predecesores del índice $j$ en la permutación $\pi$.

El costo computacional del cálculo de estos valores es alto, por lo que la librería emplea diferentes mecanismos como aproximación numérica para hallarlos además de contemplar y aprovechar de la arquitectura del modelo en cuestión para hallar los valores de una manera más eficiente.

La librería SHAP usa *Explainers* para calcular los valores SHAP. La librería tiene una variada selección de *Explainers*. Varios son *model agnostic* lo cual significa que funcionan para cualquier modelo, pero otros están diseñados específicamente para un tipo de modelo particular.

Algunos de estos *Explainers* son:
- ExactExplainer
- PermutationExplainer
- KernelExplainer
- DeepExplainer

La elección del *Explainer* no sólo determina la forma computacional en la que los valores SHAP son calculados, sino también la disponibilidad de algunas gráficas.

In [ ]:
display_question('shap_axioms')

Como este modelo es de clasificación, para poder "aumentar" las pequeñas variaciones que el modelo produce, se debe usar la escala logaritmica. La siguiente función encapsula los siguientes pasos en una sola función:
1. Normalizar los datos de entrada.
2. Hacer inferencia usando el modelo entrenado
3. Aplicar escala logarítmica

Esta función encapsuladora será la que se le pasa al *Explainer* de SHAP.

In [ ]:
def nn_predict(X):
  df_scaled = normalize(X)
  pred = model.predict(df_scaled, verbose=0, batch_size=10_000).flatten()
  return np.log(pred)

Una de las maneras de elegir un *Explainer* adecuado, es usando la clase `Explainer`que tiene la librería. Esta clase analiza la variable de entrada y la arquitectura de la función o mdelo que recibe para elegir un *Explainer* adecuado.

En la siguiente celda, se usa la clase `Explainer`, la cual recibe la función `nn_predict`como `model`.

In [ ]:
explainer = shap.Explainer(model=nn_predict, masker=X_train)

Usando `__class__` podemos ver el *Explainer* concreto que SHAP eligió para el modelo.

In [ ]:
explainer.__class__

En este caso, la librería elige el `ExactExplainer` para el cálculo de los valores de SHAP.

Este `Explainer` es eficiente para calcular los valores de SHAP para aquellos modelos que tienen menos de $15$ features. Este `Explainer` reduce la cantidad de llamadas a la función de evaluación ordenando los datos para reducir las diferencias secuenciales.

Finalmente, se deben calcular los valores SHAP para una o múltiples observaciones. Para ello se usa el `explainer`quien recibe los datos de entrada del modelo y se devuelven los valores de SHAP para cada observación.

Este proceso es bastante demandante computacionalmente, por lo tanto, sólamente se va a pasar una **fracción** del **test set** para el cálculo de los valores SHAP.

Para este caso, se usarán únicamente los primeros 1000 datos del set de prueba.

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (~ 1 linea)

## FIN DE SU CÓDIGO

<a name='3-4'></a>
### 3.4 - Gráficas Locales

SHAP ofrece la posibilidad de analizar el peso e importancia de las entradas de una observación específica.

Para este modelo, se explorarán dos gráficas para esto:
- Waterfall
- ForcePlot

Ambas gráficas están centradas en un *valor base*, el cual formalmente es el valor esperado del modelo. Sin embargo, este *valor base* se puede calcular como el promedio aritmético de los valores de las predicciones que el *Explainer* recibe.

Dependiendo el tipo de modelo que se use, existen otras gráficas que se pueden usar. Por ejemplo, en aquellos modelos en los que se analiza una imágen como entrada, existe una gráfica que muestra detalles específicos de la imágen que hacen que afecte la predicción del modelo.

#### Waterfall

Para generar la gráfica de tipo **Waterfall** se requiere llamar a la función `shap.plots.waterfall` pasando los valores de SHAP de la observación. Para este ejemplo usaremos los valores de SHAP almacenados en el índice 0.

Para que el autocalificador funcione correctamente, hay que agregar `show=False`dentro de los argumentos de la función. Esto hace que la gráfica no se muestre inmediatamente y sea posible hacer ajustes a la gráfica y/o guardarla en un archivo.
La linea `plt.show()`es la encargada de mostrar la gráfica.

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (~ 1 linea)

## FIN DE SU CÓDIGO

plt.savefig('generated_images/waterfall_1.png')
plt.show()

In [ ]:
tests.test_waterfall_plot_classification()

#### ForcePlot

Esta gráfica requiere JavaScript para funcionar, por lo tanto se inicializa mediante la función `shap.initjs()`.

Para generar la gráfica, se llama a la función `shap.plots.force` pasando los valores de SHAP de la observación. Igual que en la gráfica de tipo Waterfall, usaremos los valores de SHAP almacenados en el índice 0.

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0])

Como se puede observar, ambas gráficas muestran la misma información de una manera distinta. Sin embargo, podemos ver las variables que hacen que la predicción del modelo aumente representadas con el color rosado-rojo y las variables que hacen que la predicción del modelo disminuya representadas con el color azul. También se puede apreciar el peso de cada variable en la predicción final del modelo.

In [ ]:
display_question('wat_class_0')

In [ ]:
display_question('wat_class_1')

In [ ]:
display_question('wat_class_2')

<a name='3-5'></a>
### 3.5 - Gráficas Globales

SHAP también ofrece la posibilidad de analizar el peso e importancia de las variables de entrada en el modelo de forma general.

Existen múltiples gráficas disponibles dependiendo del modelo. Para este modelo particular, se explorarán las siguientes gráficas:

- Bar
- Beeswarm

#### Bar

Para generar la gráfica de tipo **Bar** se requiere llamar a la función `shap.plots.bar` pasando todos los valores de SHAP disponibles.

Para que el autocalificador funcione correctamente, hay que agregar `show=False`dentro de los argumentos de la función. Esto hace que la gráfica no se muestre inmediatamente y sea posible hacer ajustes a la gráfica y/o guardarla en un archivo.
La linea `plt.show()`es la encargada de mostrar la gráfica.

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (~ 1 linea)

## FIN DE SU CÓDIGO

plt.savefig('generated_images/bar_1.png')
plt.show()

In [ ]:
tests.test_bar_plot_classification()

Esta gráfica muestra un promedio de la contribución de cada componete (el promedio de su valor de SHAP). En general, si el valor es positivo y la barra es de color rosado-rojo, esto indica que este componente hace que la predicción aumente. En caso contrario, si el valor es negativo y la barra es de color azul, el componente hace que la predicción disminuya.

In [ ]:
display_question('bar_class_0')

#### Beeswarm

Para generar la gráfica de tipo **Beeswarm** se requiere llamar a la función `shap.plots.beeswarm` pasando todos los valores de SHAP disponibles.

Para que el autocalificador funcione correctamente, hay que agregar `show=False`dentro de los argumentos de la función. Esto hace que la gráfica no se muestre inmediatamente y sea posible hacer ajustes a la gráfica y/o guardarla en un archivo.
La linea `plt.show()`es la encargada de mostrar la gráfica.

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (~ 1 linea)

## FIN DE SU CÓDIGO

plt.savefig('generated_images/beeswarm_1.png')
plt.show()

In [ ]:
tests.test_beeswarm_plot_classification()

Esta gráfica brinda bastante información pues nos muestra la distribución de las contribuciones (eje X) de cada componente (eje Y) de acuerdo a su valor real (color de cada punto). Cada punto correspode a una observación particular para el que se le calcularon sus valores de SHAP.

In [ ]:
display_question('beeswarm_class_0')

In [ ]:
display_question('beeswarm_class_1')

<a name='4'></a>
## 4 - Modelo de regresión

El dataset para esta sección relaciona algunos datos médicos de personas y el costo de su seguro médico.


In [ ]:
df = pd.read_csv("insurance.csv")
df

In [ ]:
# Almacenamos los nombres de las columnas en las variables x y y
y, x = df.columns[-1], list(df.columns[:-1])

<a name='4-1'></a>
### 4.1 - Exploración del Dataset y Pre-Procesamiento

La siguiente celda da un resumen de los datos del dataframe.

In [ ]:
df.describe()

El primer paso del preprocesamiento es dividir los datos en train y test. Para esto usaremos la función`train_test_split` incluida en la librería `sklearn`. Haremos una división 90% train, 10% test.

In [ ]:
train, test = train_test_split(df, test_size=0.1, random_state=30)

In [ ]:
x_train = train.drop('charges', axis=1)
y_train = train['charges']
x_test = test.drop('charges', axis=1)
y_test = test['charges']

Como este dataset tiene columnas con datos categóricos, hay que codificar esta información en algún formato numérico, para lo cual se usará One-Hot Encoding. En este caso, vamos a usar la función de `Pandas`, `pd.get_dummies` la cual aplica One-Hot Encoding a las variables especificadas. Esta función lo que hace es para cada variable, si esta es binaria, el nuevo nombre es la variable original seguido por la primera categoría; si la variable no es binaria, se hace una nueva columna por cada categoría. La siguiente celda aplica One-Hot Encoding tanto al train set como al test set.

In [ ]:
x_train = pd.get_dummies(x_train, columns=['sex', 'smoker', 'region'], drop_first=True, dtype=float)
x_test = pd.get_dummies(x_test, columns=['sex', 'smoker', 'region'], drop_first=True, dtype=float)

# Ensure columns are aligned between train and test sets after one-hot encoding
train_cols = list(x_train.columns)
test_cols = list(x_test.columns)

for col in train_cols:
    if col not in test_cols:
        x_test[col] = 0
for col in test_cols:
    if col not in train_cols:
        x_train[col] = 0

x_test = x_test[train_cols] # Reorder test columns to match train columns

El siguiente paso es aplicar una normalización para asegurarnos que los datos estén dentro del intervalo $(-1, 1)$. Esto ayuda con la estabilidad numérica y a mantener los pesos del modelo dentro de rangos numéricamente estables.

La siguiente celda aplica la normalización que definimos anteriormente.

In [ ]:
X_train = normalize(x_train)

<a name='4-2'></a>
### 4.2 - Construcción y entrenamiento del modelo

La arquitectura de este modelo es la siguente:

- Primera capa oculta con 32 neuronas y función de activación `Relu`
- Segunda capa oculta con 64 neuronas y función de activación `Relu
- Capa de salida con función de activación lineal.

- Se utiliza el optimizador `Adam`
- Se utiliza la función de pérdida de mínimos cuadrados `mse`
- Se utiliza la función de error absoluto medio `mae` como métrica.

In [ ]:
# Define the input layer
input_layer = Input(shape=(x_train.shape[1],)) # Adjust input shape to match the data's features

# Add hidden layers
hidden_layer_1 = Dense(64, activation='relu')(input_layer) # Example number of neurons
hidden_layer_2 = Dense(32, activation='relu')(hidden_layer_1) # Example additional layer

# Add the output layer
output_layer = Dense(1, activation='linear')(hidden_layer_2) # Output layer for regression, linear activation

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', # Specify optimizer
              loss='mse', # Change loss function to Mean Squared Error for regression
              metrics=['mae']) # Change metric to Mean Absolute Error for regression

# Print the model summary
model.summary()

Ahora se entrena el modelo.

Sin embargo, para asegurarnos que la autocalificabilidad funcione correctamente, se va a cargar un modelo pre-entrenado. En caso de querer entrenar el modelo, basta con comentar la segunda linea y des-comentar la primera linea de la siguiente celda.

In [ ]:
# model.fit(x_train, y_train, epochs=5)
model = keras.saving.load_model("model2.keras")

<a name='4-3'></a>
### 4.3 - SHAP y Explainers

Como este modelo es de regresión, no es necesario aplicar escala logarítmica a las predicciones del modelo. Sin embargo, igual se deben normalizar los datos de prueba antes de ser enviados al modelo para la inferencia.

La siguiente celda aplica la normalización.

In [ ]:
def nn_predict(X):
  df_scaled = normalize(X)
  pred = model.predict(df_scaled, verbose=0, batch_size=10_000).flatten()
  return pred

En la siguiente celda, se usa la clase Explainer, la cual recibe la función nn_predict como model.

In [ ]:
explainer = shap.Explainer(model=nn_predict, masker=X_train)

Usando `__class__` podemos ver el `Explainer` que SHAP eligió para el modelo

In [ ]:
explainer.__class__

Podemos evidenciar que para este modelo también se eligió el `ExactExplainer`.

Otros explainers que SHAP puede elegir (que también se pueden usar manualmente) son:
- DeepExplainer: Optimizado para modelos de aprendizaje profundo. Es una versión mejorada de DeepLIFT (DeepSHAP) en donde los valores de SHAP son aproximados usando una selección de datos de muestra.
- KernelExplainer: Este explainer puede ser usado para explicar cualquier función. Aproxima los valores de SHAP mediante una regresión lineal especial.
- PermutationExplainer: Este explainer, el cual es model agnostic (es decir que funciona para cualquier mdoelo), aproxima los valores de SHAP mediante la iteración de múltiples permutaciones de los datos de entrada.
- TreeExplainer: Este explainer está diseñado para los modelos basados en árboles. Este explainer usa múltiples suposiciones en cuanto a dependencia de features.


Más información de los explainers se puede encontrar en la [documentación oficial de la librería](https://shap.readthedocs.io/en/latest/api.html#explainers).

In [ ]:
display_question('explainer_choice')

Finalmente, se deben calcular los valores SHAP para una o múltiples observaciones. Para ello se usa el explainer quièn recibe los datos de entrada del modelo y se devuelven los valores de SHAP para cada observación.

Aunque este proceso es bastante demandante computacionalmente, el test set de este modelo es bastante pequeño, por lo que podemos calcular los valores de SHAP para todas las observaciones del test set.

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (~ 1 linea)

## FIN DE SU CÓDIGO

<a name='4-4'></a>
###4.4 - Gráficas Locales

Para este modelo, se explorarán dos gráficas locales:

- Waterfall
- ForcePlot

#### Waterfall

Para esta gráfica se debe pasar los valores de SHAP para una sóla observación. En este caso, se usarán los valores de SHAP almacenados en el índice $0$.

Para que el autocalificador funcione correctamente, hay que agregar `show=False`dentro de los argumentos de la función. Esto hace que la gráfica no se muestre inmediatamente y sea posible hacer ajustes a la gráfica y/o guardarla en un archivo.
La linea `plt.show()`es la encargada de mostrar la gráfica.

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (~ 1 linea)

## FIN DE SU CÓDIGO

plt.savefig('generated_images/waterfall_2.png')
plt.show()

In [ ]:
tests.test_waterfall_plot_regression()

####ForcePlot

Esta gráfica requiere cargar JavaScript en la celda.

Para esta gráfica se usarán los mismos valores de SHAP que la gráfica anterior.

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0])

In [ ]:
display_question('wat_reg_0')

In [ ]:
display_question('wat_reg_1')

<a name='4-5'></a>
###4.5 - Gráficas Globales

Para este modelo, se explorarán las siguientes gráficas globales:

- Bar
- Beeswarm

#### Bar

La gráfica de tipo **Bar** requiere todos los valores de SHAP disponibles.

Para que el autocalificador funcione correctamente, hay que agregar `show=False`dentro de los argumentos de la función. Esto hace que la gráfica no se muestre inmediatamente y sea posible hacer ajustes a la gráfica y/o guardarla en un archivo.
La linea `plt.show()`es la encargada de mostrar la gráfica.

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (~ 1 linea)

## FIN DE SU CÓDIGO

plt.savefig('generated_images/bar_2.png')
plt.show()

In [ ]:
tests.test_bar_plot_regression()

In [ ]:
display_question('bar_reg_0')

In [ ]:
display_question('bar_reg_1')

#### Beeswarm

La gráfica de tipo **Beeswarm** requiere todos los valores de SHAP disponibles.

Para que el autocalificador funcione correctamente, hay que agregar `show=False`dentro de los argumentos de la función. Esto hace que la gráfica no se muestre inmediatamente y sea posible hacer ajustes a la gráfica y/o guardarla en un archivo.
La linea `plt.show()`es la encargada de mostrar la gráfica.

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (~ 1 linea)

## FIN DE SU CÓDIGO

plt.savefig('generated_images/beeswarm_2.png')
plt.show()

In [ ]:
tests.test_beeswarm_plot_regression()

In [ ]:
display_question('beeswarm_reg_0')

In [ ]:
display_question('beeswarm_reg_1')

<a name='5'></a>
##5 - Hazlo Tu Mismo (Ungraded)

Para los siguientes datasets, sigue el mismo flujo que se ha trabajado en el notebook para poder hacer interpretabilidad del modelo que se entrenará.

El siguiente dataset reúne transacciones con tarjetas de crédito que se realizaron en Europa en un fragmento de tiempo durante Septiembre de 2013. Casi toda la información está codificada, con excepción del valor de la compra, y el momento de la transacción. El objetivo es predecir si la transacción es fraudulenta.

In [ ]:
df = pd.read_csv("creditcard_31ft.csv")
df

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (Puede usar más celdas de código)



El siguiente dataset reúne datos de vehiculos y sus propietarios en Estados Unidos. El objetivo es predecir si la persona hizo un reclamo a su seguro.

In [ ]:
df = pd.read_csv("Car_Insurance_Claim.csv")
df

In [ ]:
## ESCRIBA SU CÓDIGO AQUI (Puede usar más celdas de código)



<a name='6'></a>
##6 - Bibliografía
- Actuarial-data-science/Tutorials: Code for the Actuarial Data Science Tutorials published at https://actuarialdatascience.org. https://github.com/actuarial-data-science/Tutorials/blob/master/14%20-%20SHAP/shap_tutorial.ipynb
- Shap/Shap: A game theoretic approach to explain the output of any machine learning
model. https://github.com/shap/shap?tab=readme-ov-file
- Lundberg, Scott M. ; Lee, Su-In: A Unified Approach to Interpreting Model Predic-
tions. Curran Associates, Inc., 2017
- Mayer, Michael ; Meier, Daniel ; Wuthrich, Mario: SHAP for Actuaries: Explain
any Model. En: SSRN Electronic Journal (2023), 03, p. 25
- Nasteski, Vladimir: An overview of the supervised machine learning methods. En:
HORIZONS.B 4 (2017), 12, p. 51–62
- Ng, Andrew ; Ma, Tengyu. CS229 Lecture Notes. 2023
- Norvig, Stuart J. Russell P.: Artificial Intelligence. A Modern Approach. Pearson,
Upper Saddle River, 2020. – ISBN 978–0134610993
- Yadav, Amit. SHAP Values Explained. https://medium.com/biased-algorithms/shap-values-explained-08764ab16466
- Oliveira, Willian. Healthcare Insurance https://www.kaggle.com/datasets/willianoliveiragibin/healthcare-insurance?resource=download
- Roy, Sagnik. Car Insurance Data https://www.kaggle.com/datasets/sagnik1511/car-insurance-data/data
- ULB, Machine Learning G. Credit Card Fraud Detection https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data